In [1]:
import cv2
import numpy as np
from collections import deque #bu fonksiyon sayesinde listeleme işlemi yapar

In [2]:
vid = cv2.VideoCapture(0)

In [3]:
kernel = np.ones((5,5) , np.uint8)

In [4]:
lower_blue = np.array([100,60,60])
upper_blue = np.array([140,255,255])

In [5]:
blue_points = [deque(maxlen = 512)] # mavi noktaları saklıcaz
green_points = [deque(maxlen = 512)] # mavi noktaları saklıcaz
red_points = [deque(maxlen = 512)] # mavi noktaları saklıcaz
yellow_points = [deque(maxlen = 512)] # mavi noktaları saklıcaz



In [6]:
blue_points

[deque([], maxlen=512)]

In [7]:
# renklerin index değerleri
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

In [ ]:
color = [(255,0,0),(0,255,0),(0,0,255),(0,255,255)] # renk değerleri listesi
color_index = 0
paintWindow = np.zeros((471,636,3)) +255 # cizim yapılacak pencereyi beyaz ve 3 kanallı yaptık

# paintwindow numpy değişkenine cizim için kutu olusturulur
paintWindow = cv2.rectangle(paintWindow(40,1),(140,65),(0,0,0),2)
paintWindow = cv2.rectangle(paintWindow(160,1),(255,65),color[0],-1)
paintWindow = cv2.rectangle(paintWindow(275,1),(370,65),color[1],-1) 
paintWindow = cv2.rectangle(paintWindow(40,1),(485,65),color[2],-1) 
paintWindow = cv2.rectangle(paintWindow(505,1),(600,65),color[3],-1)

font = cv2.FONT_HERSHEY_SIMPLEX
#paintwindow penceresindeki kutuların içine yazı yazılır
cv2.putText(paintWindow,"CLEAR ALL",(49,33),font,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.putText(paintWindow,"BLUE",(185,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
cv2.putText(paintWindow,"GREEN",(298,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
cv2.putText(paintWindow,"RED",(420,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
cv2.putText(paintWindow,"YELLOW",(520,33),font,0.5,(255,255,255),2,cv2.LINE_AA)

cv2.namedWindow("Paint") #  paint adlı pencere

while 1:
    
    ret , frame = vid.read()
    frame = cv2.flip(frame , 1)
    
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    #olusturulan framelerde kutu cizimi 
    frame = cv2.rectangle(frame,(40,1),(140,65),(0,0,0),2)
    frame = cv2.rectangle(frame,(160,1),(255,65),color[0],-1)
    frame = cv2.rectangle(frame,(275,1),(370,65),color[1],-1)
    frame = cv2.rectangle(frame,(390,1),(485,65),color[2],-1)
    frame = cv2.rectangle(frame,(505,1),(600,65),color[3],-1)
    
    font = cv2.FONT_HERSHEY_SIMPLEX # yazı fontu tanımlanır 
    #kutuların  içine yazı yazma
    cv2.putText(frame,"CLEAR ALL",(49,33),font,0.5,(0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"BLUE",(185,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"GREEN",(298,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"RED",(420,33),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"YELLOW",(520,33),font,0.5,(255,255,255),2,cv2.LINE_AA)

    if ret is False:
        break
                                                  # frame'deki mavi nesneyi ikili maske yapar
    mask = cv2.inRange(hsv,lower_blue,upper_blue) #iki değer arasındaki bölümü beyaz yapar

    mask = cv2.erode(mask,(5,5),iterations =2) #  Maske üzerindeki beyaz alanları (1 olan yerleri) küçültmek
                                                # Küçük beyaz noktacıkları silmek için kullanılır.
                                                    
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,(5,5)) #Beyaz nesnelerin içindeki küçük siyah delikleri veya etrafındaki beyaz gürültüleri temizlemeye yardımcı olur.
  
    mask = cv2.dilate(mask,(5,5),iterations = 1) # görüntümüzdeki az sayıdaki 0'ları 1 yapar
                                                 # Beyaz alanları (1 olan yerleri) genişletmek.
                                                 # inceltilmis nesneyi kalınlastırır
                                                 # Objenin daha düzgün ve kesintisiz görünmesini sağlar.
  
    contours,_ =cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #görüntüdeki kontourlar bulunur
    center = None
    if len(contours) > 0:
        max_contours = sorted(contours, key = cv2.contourArea, reverse=True)[0] #maskelenen nesnenin alanı en büyük olan contour'u daire içine alır
                                                                                #framedeki  mavi olarak algılanan islem yapmıcamız kucuk konturları almamak için en büyük olanı secme işlemi yapılır
        ((x,y),radius) = cv2.minEnclosingCircle(max_contours) # en buyuk contour'u  sınırlayan cember bulunur
                                                              # kordinat ve cap değerleri döndürür
                                                            
        cv2.circle(frame,(int(x),int(y)),int(radius),(255,255,0),3) # bulunan kordinatlar ve capa göre cember cizilir

        M = cv2.moments(max_contours) # cember'in merkez noktalarına erismek için kullanılır
        center = (int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"])) #cemberin merkez noktasını buluruz

        # merkezin bulunduğu noktamıza göre renk değişimi yapılır
        if center[1] <= 65: # cember merkezimizin y kordinatı 65'ten kucukse 
            if 40<=center[0]<=140: #merkezimizin x kordinat değeri 40 ile 140 arası ise
                
                blue_points = [deque(maxlen=512)]
                green_points = [deque(maxlen=512)]
                red_points = [deque(maxlen=512)]
                yellow_points = [deque(maxlen=512)]

                blue_index=0
                green_index=0
                red_index=0
                yellow_index=0

                paintWindow[67:,:,:]=255 #tüm ekran beyaz olur 

            elif 160<=center[0]<=255: # x kordinatı 160 255 5arası ise color_index
                color_index = 0

            elif 275<=center[0]<=370: # x kordinat değeri 275 370 araıs değer alırsa
                color_index = 1 # yesil renk alır

            elif 390<=center[0]<=485: # x kordinat değeri 390 ile 485 değeri arsaında ise
                color_index = 2 # kırmızı  renk alınır

            elif 505<=center[0]<=600:# x kordinat değeri 505 ile 600 arası ise 
                color_index = 3 #sarı renk alınır

        else: # eğer y ekseni 65'ten büyükse 
            if color_index == 0: # color_index 0 ilse mavi renkte cizim yapılır
                blue_points[blue_index].appendleft(center) # color_indexin tuttuğu renk değerinde renk noktaları ard arda eklenir
                
            elif color_index == 1:  # inde x1 ise  yesil cizim yapılır
                green_points[green_index].appendleft(center)
                
            elif color_index == 2: #iki ise kırmızı cizim 
                red_points[red_index].appendleft(center)
                
            elif color_index == 3: # 3 ie sarı cizim 
                yellow_points[yellow_index].appendleft(center)

    else: # biren fazla contour yoksa 
        blue_points.append(deque(maxlen=512))
        blue_index+=1
        
        green_points.append(deque(maxlen=512))
        green_index+=1
        
        red_points.append(deque(maxlen=512))
        red_index+=1
        
        yellow_points.append(deque(maxlen=512))
        yellow_index+=1

    points = [blue_points,green_points,red_points,yellow_points]
    
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1,len(points[i][j])):
                if points[i][j][k-1] is None or points[i][j][k] is None:
                    continue
                
                cv2.line(frame,points[i][j][k-1], points[i][j][k], color[i], 2)
                cv2.line(paintWindow,points[i][j][k-1], points[i][j][k], color[i], 2)
    
    cv2.imshow("frame",frame)
    cv2.imshow("paintwindow",paintWindow)
    
    if cv2.waitKey(3) & 0xFF == ord("q"):
        break
    
vid.release()
cv2.destroyAllWindows()    
